In [26]:
import numpy as np
import pandas as pd

df = pd.read_csv("fr-2017-pssd-compendium-20180320-utf8.csv")
df.head()

,Secteur,Nom de famille,Prénom,Traitement versé,Avantages imposables,Employeur,Poste,Année civile
0,Autres employeurs du secteur public,Coleridge,Peter,"$250,000.00","$25,749.50",Big Brothers Big Sisters of Canada,National President and Chief Executive Officer,2017
1,Autres employeurs du secteur public,Chater,W. Matthew,"$100,877.00","$2,778.27",Big Brothers Big Sisters of Canada,"National Vice-President, Service and Organizat...",2017
2,Autres employeurs du secteur public,Mcintyre,Mary Ellen,"$113,847.58","$19,015.00",Centre For Spanish Speaking Peoples,Legal Director,2017
3,Autres employeurs du secteur public,Marlin,Susan,"$248,068.75",$258.48,Clinical Trials Ontario,President and Chief Executive Officer,2017
4,Autres employeurs du secteur public,Frampton,Peter,"$165,033.85","$10,788.30",Learning Enrichment Foundation,Executive Director,2017


In [27]:
df["salary"] = df['Traitement versé'
                           ].replace( '[\$,]','', regex=True
                           ).astype(float
                           ) + df['Avantages imposables'
                                 ].replace( '[\$,]','', regex=True
                                 ).astype(float)

In [28]:
# Standardize strings to ascii form
from unidecode import unidecode 
df['name'] = df['Prénom'].apply(lambda n : unidecode(n).lower())

In [29]:
genders =  pd.read_csv("us-likelihood-of-gender-by-name-in-2014.csv")
genders.name = genders.name.str.lower()
genders.head()

,sex,name,gender.prob
0,F,elaine,1.0
1,F,cathy,1.0
2,F,heidi,1.0
3,F,vicki,1.0
4,F,melinda,1.0


In [30]:
df = df.merge(genders, on="name")
df.sex = df.sex.map({'M': 1, "F":0}).astype('int8')
df = df.drop(["Nom de famille", "Traitement versé", 
              "Avantages imposables", "Année civile",
              "Prénom"], 1)

In [31]:
import statsmodels.api as sm
import numpy as np

X = sm.add_constant(df.sex)
sm.OLS(np.log(df.salary), X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 salary   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     1105.
Date:                Tue, 10 Jul 2018   Prob (F-statistic):          5.80e-241
Time:                        23:07:50   Log-Likelihood:                 11362.
No. Observations:              101732   AIC:                        -2.272e+04
Df Residuals:                  101730   BIC:                        -2.270e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.7026      0.001   1.13e+04      0.000      11.701      11.705
sex            0.0456      0.001     33.239      0.000       0.043       0.048
==============================================================================
Omnibus:                    42964.745   Durbin-Watson:                   1.648
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           219409.558
Skew:                           2.018   Prob(JB):                         0.00
Kurtosis:                       8.956   Cond. No.                         2.83
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
X = sm.add_constant(df.sex
      ).join(
        pd.get_dummies(df.Employeur, 
                       sparse=True, # This is necessary due to data size!
                       drop_first=True)
    )

sm.OLS(np.log(df.salary), X).fit().summary()

NameError: name 'gc' is not defined

# Meddling around with gendered names

In [ ]:
import gensim
import gensim.downloader as api
from sklearn import cluster

model = api.load('glove-twitter-100')

[--------------------------------------------------] 0.2% 0.6/387.1MB downloaded

In [66]:
names = df.name.unique()
# pre allocate memory for efficiency
nameVectors = np.zeros((len(names),
                        # length of word vector
                        len(model[names[0].lower()])))
oovNames = set()
for i in range(len(names)):
    try:
        nameVectors[i] = model[names[i]]
    except KeyError as ke:
        oovNames.update(names[i])
        
genderClusters = cluster.k_means(nameVectors, 2)[1]

In [67]:
dn = pd.DataFrame(nameVectors)
dn['name'] = names
dn = dn.merge(genders, on="name")
dn['cluster'] = genderClusters
dn

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,name,sex,gender.prob,cluster
0,-0.259020,0.133320,0.252230,-0.493660,-0.352480,-0.693030,-0.453840,0.823390,-0.841220,-0.924240,...,-0.004684,0.177440,0.088567,0.716740,-0.093850,-1.052800,peter,M,0.999235,1
1,-0.034050,-0.096682,0.329590,0.120860,-0.429080,-0.288000,-0.395850,0.517840,-0.866500,-0.107060,...,0.464580,-0.121720,0.047698,-0.846230,0.189150,-0.393300,susan,F,0.999294,1
2,-0.305640,-0.286850,0.173680,-0.691890,-0.522460,-0.700410,-0.358600,-0.398770,-0.851560,-0.590860,...,-1.148900,0.210750,-0.024062,-0.305170,0.244650,-0.213890,therese,F,1.000000,0
3,0.154380,0.065990,0.549620,-1.060700,-0.830870,-0.956520,0.497920,0.611810,-1.012700,-0.483340,...,-0.834380,-0.161770,-0.527390,-0.519520,-0.051767,-0.784180,daniel,M,0.997306,1
4,-0.626250,-0.317810,0.181360,-0.078804,-0.261210,-0.974440,-0.846960,-0.078957,-0.178970,-0.724990,...,-0.370940,-0.546100,-0.368030,-0.077725,-0.131660,-1.060600,deborah,F,0.999481,1
5,-0.771520,-0.051431,0.487570,-0.880540,-0.451090,-0.626190,-0.294300,-0.004366,-0.882240,-0.690150,...,-0.841600,-0.056302,0.300910,-0.373660,0.512570,-0.037283,shirley,F,0.994464,1
6,0.164650,0.530480,0.406650,0.462680,-0.446660,-0.927010,-0.474970,-0.241760,0.738890,0.050296,...,-0.098318,0.072908,-0.265090,0.199030,0.066358,-0.620430,sue,F,0.999374,1
7,-0.968060,-0.146050,0.247910,-0.109010,-0.456560,0.244070,-0.237480,0.488390,-0.399310,-0.138480,...,-0.770020,0.263650,-0.110720,-0.290730,0.282550,-0.400610,colette,F,1.000000,0
8,0.089727,-0.511270,-0.370370,-1.166300,-0.333180,-0.206030,-0.528500,-0.170640,-0.962210,-1.278000,...,-0.149400,-0.660070,-0.786160,-0.622410,0.198610,0.092877,jamison,M,0.974183,0
9,-0.459910,-0.299430,0.710660,-0.574780,-0.595010,-0.414660,0.026063,0.757730,-0.731220,-0.495100,...,-0.280860,-0.189740,-0.087563,-0.175840,-0.854770,-0.764760,michael,M,0.995736,1
